## get all of the runs for global uncertainty

UQTK requires 3 inputs: 
- a text file containing all of the perturbed values
- a text file containing all of the ranges for the perturbed values
- a text file containing the outputs that you'd like to get sensitivity to (for us, meoh tof, h2o tof)

There is a split for training vs validation, I am just goint to do 75% to start idk what best practices are

In [5]:
import pandas as pd
import csv
import pickle
import os
import copy
from torch.quasirandom import SobolEngine
from rmgpy.data.kinetics.database import KineticsDatabase
from rmgpy.data.surface import MetalDatabase
from rmgpy.molecule import Molecule
from rmgpy.data.thermo import ThermoDatabase
from rmgpy import constants
from tqdm import tqdm  # this is for the progress bar cause copying stuff takes a while
import numpy as np
import math
import csv
import glob
import re

In [7]:
unc_folder = "/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/"
conda_path = unc_folder + "conda/"
RMG_base_folder = unc_folder + "RMG-Py/"
RMG_db_folder = unc_folder + "RMG-database/"
conda_path = unc_folder + "conda/"
expt_yaml_file = ""
output_path  = "/scratch/blais.ch/methanol_results_2022_07_19/"

In [8]:
# we need to know which runs failed. use script from the postprocessing notebook that 
# checks for the objective function file
# we only want the ct_analysis files that were completed most recently, 
# which will have an objective function file with them
obj_func_filename = "objective_function_log_ct_analysis.txt"
ct_filename = "ct_analysis.csv"
ct_data_files = glob.glob(os.path.join(output_path,f"run_*/cantera/{ct_filename}"))

obj_func_files = [i.replace(ct_filename, obj_func_filename) for i in ct_data_files]
obj_func_dict = {}
model_size_dict = {}
pattern = re.compile('run_+\d')
for file in obj_func_files:
    with open(file, "r") as f:
        line = f.readline()
        path,obj_func = line.split(":")
        
        # find the string for the run number 
        pattern = re.compile('run_\d{4}')
        match = re.search('run_\d{4}',file)
        run_num = int(match.group(0).replace('run_', ""))
        
        obj_func_dict[run_num] = float(obj_func)
        
    # access log file to get number fo species    
    new_addr = file.replace(f"cantera/{obj_func_filename}", "RMG.log")
    with open(new_addr, "r") as f:
        for line in f.readlines():
            if re.search('The final model core has', line, re.I):
                match_spec = re.search('[0-9]+ species',line)
                spec_num = int(match_spec.group(0).replace(" species", ""))
                
                match_reac = re.search('[0-9]+ reactions',line)
                reac_num = int(match_reac.group(0).replace(" reactions", ""))

                model_size_dict[run_num] = (spec_num,reac_num)

# generate list of passed and failed runs
# get number of folders

total_run_folders = glob.glob(os.path.join(output_path,"run_*/"))
all_runs = list(range(0,len(total_run_folders)))

passed_runs = list(obj_func_dict.keys())
passed_runs.sort()


# use a set to get the difference between passed and failed runs 
failed_runs_set = set(all_runs).difference(passed_runs)
failed_runs = list(failed_runs_set)

closest = min(obj_func_dict, key=lambda y: abs(obj_func_dict[y]))
print(closest,obj_func_dict[closest])

7326 0.10475583648188946


In [24]:
num_passed = len(passed_runs)
num_train = round(0.75*len(passed_runs))
num_val = num_passed-num_train
print("Training: ", num_train,"\nValidation: ", num_val)

Training:  5258 
Validation:  1752


# sanitize inputs
first, load in the perturbed values  
- the sobol map pickle has the value of the perturbed parameters. load this, then scale the a factors so we're looking at their exponent? 
- everything else should be okay I think? 

In [26]:
sobol_path = f"{output_path}sobol_map.pickle"
with open(sobol_path, "rb") as input_file:
    perturb_values = pickle.load(input_file)

In [27]:
# reconstruct dict so it is {key:list of values}
pert_val_san = {}
for param, values in perturb_values.items():
    # extract a list of just perturbed values, since we 
    # have a tuple with that and other metadata
    just_pert = values[1].tolist()
    
    # check for A factors that are not sticking coefficient
    # record the value at
    if min(just_pert) > 1.0 and param.endswith("/A"): 
        just_pert = [math.log(i,10) for i in just_pert]
    
    pert_val_san[param] = just_pert

# re-organize so we have a list of dicts, with the perturbed values as each individual dict? 
pert_val_san_list = []
for i in range(10000):
    ind_pert_dict = {}
    for key in pert_val_san.keys():
        ind_pert_dict[key] = pert_val_san[key][i]
    
    pert_val_san_list.append(ind_pert_dict)
    

Write the values to a space separated text file. from uqtk doc: 
- input is a Nxd matrix, each row is a d-variate parameter sample (so each column is a different perturbed value?)

In [29]:
# # passed_runs
# perturb_values_copy = {}
# for run (param, val) in enumerate(perturb_values.items()):
#     if run in passed_runs:
#         perturb_values_copy
pert_val_san_list_passed = []
for run, data in enumerate(pert_val_san_list):
    if run in passed_runs:
        pert_val_san_list_passed.append(data)

len(pert_val_san_list_passed)

7010

In [30]:
csv_file = "Input_meoh.txt"
csv_columns = list(perturb_values.keys())
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, delimiter=' ',fieldnames=csv_columns)
        # we will remove the header row, seems dicy because I do not know if the 
        # values are written correctly
        # writer.writeheader()
        for data in pert_val_san_list_passed:
            writer.writerow(data)

except IOError:
    print("I/O error")

 and that is the input data. now add the ranges

In [31]:
sobol_range_path = f"{output_path}sobol_range_map.pickle"
with open(sobol_range_path, "rb") as input_file:
    perturb_range_values = pickle.load(input_file)

if all the same, grab the first item from each list to make a new dict

In [ ]:
# perturb_range_values_single = {}
# for key, value in perturb_range_values.items(): 
#     first_val_init = perturb_range_values[key][0]

#     if first_val_init[0] > 1.0 and key.endswith("/A"):
#         first_val = (math.log(first_val_init[0], 10), math.log(first_val_init[1], 10))
#     else:
#         first_val = first_val_init

#     perturb_range_values_single[key] = first_val
    
# perturb_range_values_single

In [32]:
perturb_range_values['Surface_Abstraction/Abstracting;Donating/A']

(17.505149978319906, 16.505149978319906, 18.505149978319906)

In [37]:
# # second and third value of sobol_range_map are min and max. 
# for key, value in perturb_range_values.items():
#     print("{0:f} {1:f} {2:f}\n".format(value[0], value[1], value[2]))

0.370000 0.220000 0.520000

17.505150 16.505150 18.505150

99300.000000 69300.000000 129300.000000

0.370000 0.220000 0.520000

17.505150 16.505150 18.505150

99300.000000 69300.000000 129300.000000

0.895000 0.790000 1.000000

17.505150 16.505150 18.505150

129300.000000 99300.000000 159300.000000

0.650000 0.500000 0.800000

17.505150 16.505150 18.505150

22700.000000 0.000000 45400.000000

0.370000 0.220000 0.520000

17.505150 16.505150 18.505150

99300.000000 69300.000000 129300.000000

0.895000 0.790000 1.000000

17.505150 16.505150 18.505150

129300.000000 99300.000000 159300.000000

0.895000 0.790000 1.000000

17.505150 16.505150 18.505150

129300.000000 99300.000000 159300.000000

0.650000 0.500000 0.800000

17.505150 16.505150 18.505150

22700.000000 0.000000 45400.000000

0.680000 0.530000 0.830000

17.505150 16.505150 18.505150

106100.000000 76100.000000 136100.000000

0.680000 0.530000 0.830000

17.505150 16.505150 18.505150

106100.000000 76100.000000 136100.000000

0.085

In [39]:
# now just make a text file. 
range_file = "parameter_ranges_meoh.txt"
with open(range_file, "w") as f:
    for key, value in perturb_range_values.items():
        f.write("{0:f} {1:f}\n".format(value[1], value[2]))

# get parameter names

In [40]:
pert_value_names = list(perturb_range_values.keys())

with open("parameter_names_meoh.txt", "w") as f:
    for name in pert_value_names:
        f.write(name+"\n")

# get output data and write to file

In [ ]:
ct_data_dict = {}

data_keys = [
    "RMG MeOH TOF 1/s",
    "RMG H2O TOF 1/s",
]

for file in ct_data_files:
    # find the string for the run number 
    pattern = re.compile('run_\d{4}')
    match = re.search('run_\d{4}',file)
    run_num = int(match.group(0).replace('run_', ""))
    df = pd.read_csv(file)
    data_run = df[(df['Unnamed: 0'] == 80)]
    data_dict = {}
    
    for key in data_keys: 
        output_value = float(data_run[key])
        if  output_value <=0: 
            output_value = 0
        else: 
            output_value = math.log(output_value, 10)
        data_dict[key] = output_value
        print(run_num,key,output_value,data_dict[key])
#     data_dict["run"] = run_num
    
    ct_data_dict[run_num]= data_dict
    
# sort by run
# ct_data_dict.sort(key=lambda y: y["run"])

In [5]:
dict1 = {
    "a":1,"b":9,"c":6,"d":3,"e":8,"f":7,
}
dict2 = {
    "a":3,"b":12,"c":15,"d":17,"e":2,"f":1,
}
dict3 = {
    "a":4,"b":19,"c":73,"d":2,"e":2,"f":0,
}
dicty = []
dicty.append(dict1)
dicty.append(dict2)
dicty.append(dict3)
dicty.sort(key=lambda y: y["a"])
    

## more shuffling, I need to remove the runs that are less than = 0 meoh TOF

In [43]:
pert_val_san_dict_passed = {}
for run, data in enumerate(pert_val_san_list):
    if run in passed_runs:
        pert_val_san_dict_passed[run] = data

In [44]:
pert_val_san_dict_passed_copy = pert_val_san_dict_passed.copy()
ct_data_dict_copy = ct_data_dict.copy()
for run, data in ct_data_dict.items():
    
    for key, value in ct_data_dict_copy[run].items():
        print(key,value)
        if value == 0:
            del pert_val_san_dict_passed_copy[run]
            del ct_data_dict_copy[run]
            print(run)
            break

RMG MeOH TOF 1/s -6.375404571626528
RMG H2O TOF 1/s -2.0801158826539377
RMG MeOH TOF 1/s -6.160304257782753
RMG H2O TOF 1/s -2.3925031480635215
RMG MeOH TOF 1/s -8.315201100097589
RMG H2O TOF 1/s 0
4323
RMG MeOH TOF 1/s -9.484591425787555
RMG H2O TOF 1/s -0.8196489110220005
RMG MeOH TOF 1/s -5.741662237452091
RMG H2O TOF 1/s 0
2533
RMG MeOH TOF 1/s -5.394012522124205
RMG H2O TOF 1/s -1.4550602630358411
RMG MeOH TOF 1/s -5.013941931349627
RMG H2O TOF 1/s -1.5708710968187451
RMG MeOH TOF 1/s -4.386264013104043
RMG H2O TOF 1/s 0.050707294225038134
RMG MeOH TOF 1/s -9.352657225044547
RMG H2O TOF 1/s -2.9152046477027937
RMG MeOH TOF 1/s 0
6738
RMG MeOH TOF 1/s -10.425239169363527
RMG H2O TOF 1/s -4.432148037140881
RMG MeOH TOF 1/s -8.286480039094098
RMG H2O TOF 1/s -2.4192195062471953
RMG MeOH TOF 1/s -6.07938542227344
RMG H2O TOF 1/s -1.5336880720328587
RMG MeOH TOF 1/s -2.4891539752895646
RMG H2O TOF 1/s 0.2701840311116574
RMG MeOH TOF 1/s -7.491156951761614
RMG H2O TOF 1/s -0.73265464341

In [45]:
len(ct_data_dict_copy)

6692

In [50]:
total = len(ct_data_dict_copy)
train = 5000
val = total -train 
print(train, val)

5000 1692


In [46]:
# now just make a text file. 
output_file = "outputs_meoh.txt"
with open(output_file, "w") as f:
    for key, data_keys in ct_data_dict_copy.items():
        val_str_list = []
        for value in data_keys:
            val_str_list.append("{0:e}".format(data_keys[value]))
        value_str = " ".join(val_str_list)
        f.write(f"{value_str}\n")

In [47]:
csv_file = "Input_meoh.txt"
csv_columns = list(perturb_values.keys())
try:
    with open(csv_file, 'w') as f:
        writer = csv.writer(f, delimiter=" ")
        # we will remove the header row, seems dicy because I do not know if the 
        # values are written correctly
        # writer.writeheader()
        for key,perts in pert_val_san_dict_passed_copy.items():
            data = []
            for value in perts.values():
                data.append(value)
            writer.writerow(data)

except IOError:
    print("I/O error")